I'll use this notebook to study potential LJ constituents and look into whether I can unambiguously pick out the relevant particles with dR requirements. This study should be updated to instead use pfjet.pfcand and compare directly with ljsources.

In [1]:
# python
import sys
import os
import importlib
# columnar analysis
from coffea import processor
import awkward as ak
from dask.distributed import Client
# locala
sys.path.insert(1, os.path.join(sys.path[0], '../..')) # fixme: there must be a better way to handle this...
from sidm.tools import ffschema, sidm_processor, utilities
# always reload local modules to pick up changes during development
importlib.reload(ffschema)
importlib.reload(sidm_processor)
importlib.reload(utilities)
# plotting
import matplotlib.pyplot as plt
utilities.set_plot_style()

<function <lambda> at 0x7fab52cab820>


In [2]:
# samples = [
#     "2Mu2E_100GeV_5GeV_400mm",
#     "2Mu2E_100GeV_5GeV_200mm",
#     "2Mu2E_100GeV_5GeV_40mm",
#     "2Mu2E_100GeV_5GeV_4mm",
#     "2Mu2E_100GeV_5GeV_0p4mm",
# ]

samples = [
    "2Mu2E_800GeV_1p2GeV_1p2mm",
    "2Mu2E_800GeV_0p25GeV_1p2mm",
]

# samples = [
#     "2Mu2E_1000GeV_5GeV_40mm",
#     "2Mu2E_500GeV_5GeV_40mm",
#     "2Mu2E_100GeV_5GeV_40mm",
# ]

# samples = [
#     "2Mu2E_100GeV_5GeV_4mm",
#     "2Mu2E_150GeV_5GeV_2p7mm",
#     "2Mu2E_200GeV_5GeV_2mm",
#     "2Mu2E_500GeV_5GeV_0p8mm",
#     "2Mu2E_800GeV_5GeV_0p5mm",
#     "2Mu2E_1000GeV_5GeV_0p4mm",
# ]
fileset = utilities.make_fileset(samples, "ffntuple_v4")

In [3]:
runner = processor.Runner(
    #executor=processor.IterativeExecutor(),
    executor=processor.FuturesExecutor(),
    schema=ffschema.FFSchema,
)

p = sidm_processor.SidmProcessor(
    [
        "2mu2e",
        "2mu2e_matchedLj",
    ],
    ["base_plus_gen"]
)

output = runner.run(fileset, treename="ffNtuplizer/ffNtuple", processor_instance=p)
#hists = output["out"][samples[0]]["hists"]

ch1 = "2mu2e"
ch2 = "2mu2e_matchedLj"

Output()

Output()

/opt/conda/lib/python3.9/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(


[{x: -59.6, y: 1.6, z: -178, t: 187, charge: -1, pid: 13, ... y: 0.0592, z: 2.77}}]
[0.000148, 0.000107]


concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/coffea/processor/executor.py", line 1654, in _work_function
    out = processor_instance.process(events)
  File "/home/cms-jovyan/SIDM/sidm/studies/../../sidm/tools/sidm_processor.py", line 106, in process
    h.fill(sel_objs, evt_weights)
  File "/home/cms-jovyan/SIDM/sidm/studies/../../sidm/tools/histogram.py", line 45, in fill
    fill_args = {a.name: a.fill_func(objs, self.evt_mask(objs)) for a in self.axes}
  File "/home/cms-jovyan/SIDM/sidm/studies/../../sidm/tools/histogram.py", line 45, in <dictcomp>
    fill_args = {a.name: a.fill_func(objs, self.evt_mask(objs)) for a in self.axes}
  File "/home/cms-jovyan/SIDM/sidm/studies/../../sidm/definitions/hists.py", line 175, in <lambda>
    lambda objs, mask: ak.num(derived_objs["muons_matched_genMus"](objs, 0.4))),
  File "/home/cms-jovyan/SIDM/sidm/studies/../../sidm/definitions/objects.py", line 40, in

[]
[]
[]
[]
[]
[]
[{x: -110, y: -232, z: -543, t: 601, charge: -1, pid: 13, ... y: -1.1, z: -7.68}}]
[0.000256, 0.000387]
[{x: 78.7, y: -141, z: 39.4, t: 166, charge: 1, pid: -13, ... y: -10.6, z: 3.5}}]
[0.000636, 0.000913]
[]
[]


Exception: Failed processing file: WorkItem(dataset='2Mu2E_800GeV_0p25GeV_1p2mm', filename='root://xcache//store/group/lpcmetx/SIDM/ffNtupleV4/2018/SIDM_XXTo2ATo2Mu2E_mXX-800_mA-0p25_ctau-1p2_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18DRPremix-102X_upgrade2018_realistic_v15-v1/210326_161534/0000/ffNtuple_5.root', treename='ffNtuplizer/ffNtuple', entrystart=0, entrystop=6419, fileuuid=b'P!\x85\xc0\x8eO\x11\xeb\xb1\xdeg\xb4\\\x90\xbe\xef', usermeta={})

In [ ]:
legend_entries = [s[13:] for s in samples]
nplots = 2

# LJs:
plt.subplot(2,1,1) # rows, columns, which subplot
for S in samples:  
    output["out"][S]["hists"]["matched_muon_n"][ch1,:].plot(density=False, flow='none')
plt.title("number of matched muons")
plt.legend(legend_entries, alignment="left")
#-----

# Muons:
plt.subplots(1, nplots, figsize=(nplots*12, 10))
plt.subplot(1,nplots,1) # rows, columns, which subplot
for S in samples:  # genAs_toMu_matched_lj
    output["out"][S]["hists"]["genAs_toMu_invmass"][ch1,:].plot(density=False, flow='none')
plt.title("invariant mass of muons from genAs")
plt.legend(samples)

plt.subplot(1,nplots,2)
for S in samples: # mu_mu_invmass_lowRange
    output["out"][S]["hists"]["muons_matched_muLj_invmass"][ch1,:].plot(density=False, flow='none')
plt.title("invariant mass of matched reco muons")
plt.legend(samples)
#-----

# Electrons:
plt.subplots(1, nplots, figsize=(nplots*12, 10))
plt.subplot(1,nplots,1)
for S in samples: # genAs_toE_matched_lj
    output["out"][S]["hists"]["genAs_toE_invmass"][ch1,:].plot(density=False, flow='none')
plt.title("invariant mass of electrons from genAs")
plt.legend(samples)

plt.subplot(1,nplots,2)
for S in samples: # egm_egm_invmass_lowRange | electron_invmass
    output["out"][S]["hists"]["electrons_matched_egmLj_invmass"][ch1,:].plot(density=False, flow='none')
plt.title("invariant mass of matched reco electrons")
plt.legend(samples)